In [26]:
import pandas as pd
import numpy as np
import warnings 
import matplotlib.pyplot as plt
# 라이브러리 로드

import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from scipy import stats
from scipy.stats import shapiro, anderson, kstest, jarque_bera, normaltest
from scipy.stats import bartlett
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import SMOTE

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import warnings
# 라이브러리 import 

import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error, confusion_matrix

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


import warnings 
warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Malgun Gothic'

# 한글 깨짐 방지
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'Malgun Gothic'
Raw = pd.read_csv('./csv/Tabnet_Raw.csv')
Raw.replace([np.inf, -np.inf], np.nan, inplace=True)
Raw.dropna(inplace=True)
Raw.columns

Index(['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '시가총액', '거래량회전율', '로그시가총액', '수정거래량',
       '거래량증가율', '시가총액증가율', '부실'],
      dtype='object')

In [27]:
Raw_ylabel = Raw[['회사명', '거래소코드', '회계년도', '부실']]
Raw_data = Raw[['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '시가총액', '거래량회전율', '로그시가총액', '수정거래량',
       '거래량증가율', '시가총액증가율']]

In [28]:
# IQR
def outliers_ipr_side(df, col):
    ## 1사분위수, 3사분위수 변수에 대입
    quartile_1, quartile_3 = np.percentile(df[col], [0.5, 99.5]) # np.percentile은 영상의 intensity 중 분위수를 구해주는 함수
    ## iqr 값을 대입
    iqr = quartile_3 - quartile_1
    ## 하단 경계 값, 상단 경계 값
    lower_whis = quartile_1 - (1.5 * iqr)
    upper_whis = quartile_3 + (1.5 * iqr)
    # print(lower_whis, upper_whis)
    ## 데이터프레임에서 상단 경계보다 크거나 하단의 경계보다 작은 데이터를 출력
    outliers = df[(df[col] > upper_whis) | (df[col] < lower_whis)]
    return outliers[[col]]

# outlier 값들 양끝단 합해서 1% 제거
for i in Raw_data.columns:
    a = outliers_ipr_side(Raw_data, i).index
    Raw_data.drop(a, axis=0, inplace=True)

## outlier 값들을 NaN값으로 교체후 평균으로 채우기
# outliers_dict = {}
# for col in Raw_data.columns:
#     outliers = outliers_ipr_side(Raw_data, col).index
#     outliers_dict[col] = outliers
# for col in Raw_data.columns:
#     outliers = outliers_dict[col]
#     Raw_data.loc[outliers, col] = np.NaN
#     col_mean = Raw_data[col].mean()
#     Raw_data[col] = Raw_data[col].fillna(col_mean)

merged_df = pd.merge(Raw_data, Raw_ylabel, left_index=True ,right_index=True, how='inner')
merged_df_IQR = merged_df.copy()
merged_df_IQR = merged_df_IQR[['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '시가총액', '거래량회전율', '로그시가총액', '수정거래량',
       '거래량증가율', '시가총액증가율', '부실']]
merged_df_IQR.drop_duplicates(keep='first', inplace=True)
merged_df_IQR.reset_index(drop=True, inplace=True)

In [29]:
# iqr split
train_iqr = merged_df_IQR[merged_df_IQR['회계년도'] <= 2017]
test_iqr = merged_df_IQR[merged_df_IQR['회계년도'] > 2017]
X_train = train_iqr.drop(columns=['회사명', '거래소코드', '회계년도', '부실'], axis=1)
X_test = test_iqr.drop(columns=['회사명', '거래소코드', '회계년도', '부실'], axis=1)
Y_train = train_iqr['부실']
Y_test = test_iqr['부실']
X_train_copy = X_train.copy()
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

In [30]:
X_train_1 =X_train[['로그시가총액']]
X_train = X_train[['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '거래량회전율',  '수정거래량', '거래량증가율', '시가총액증가율']]

X_test_1 =X_test[['로그시가총액']]
X_test = X_test[['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', 'DOL', 'DFL', 'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률',
       '고정비율', 'R&D비율', '채무부담비율', '거래량회전율',  '수정거래량', '거래량증가율', '시가총액증가율']]

In [31]:
# standardscaler
from sklearn.preprocessing import StandardScaler
scaler_St = StandardScaler()
scaler_St.fit(X_train)
data_scaled_MM = scaler_St.transform(X_train)
X_train_MM = pd.DataFrame(data = data_scaled_MM, columns=X_train.columns)
scaler_St.fit(X_train)
data_scaled_MM = scaler_St.transform(X_test)
X_test_MM = pd.DataFrame(data = data_scaled_MM, columns=X_test.columns)

In [32]:
# 스케일링 다음
X_train_MM = pd.concat([X_train_MM,X_train_1], axis=1)
X_test_MM = pd.concat([X_test_MM,X_test_1], axis=1)
X_train = pd.concat([X_train,X_train_1], axis=1)
X_test = pd.concat([X_test,X_test_1], axis=1)

In [33]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import BorderlineSMOTE
rus = BorderlineSMOTE(sampling_strategy=1)
X_train_MM, Y_train = rus.fit_resample(X_train_MM, Y_train)

In [34]:
# T-test
# 등분산성 확인 전 레이블 부착
Y_train = Y_train.reset_index()
X_train_MM = X_train_MM.reset_index()

Y_train.drop(columns=['index'], axis=1, inplace=True)
X_train_MM.drop(columns=['index'], axis=1, inplace=True)

x_등분산 = pd.concat([X_train_MM, Y_train], axis=1)
x_등분산

,자기자본구성비율,설비투자효율,총자본투자효율,이자보상배율(이자비용),유동비율,당좌비율,부채비율,총자본정상영업이익률,매출액정상영업이익률,매출액순이익률,...,금융비용부담률,고정비율,R&D비율,채무부담비율,거래량회전율,수정거래량,거래량증가율,시가총액증가율,로그시가총액,부실
0,0.534222,0.197471,-0.051690,-0.665609,-0.090835,-0.051761,-0.441782,-0.164152,0.051503,-0.352927,...,-0.218961,-0.260378,-0.398858,-0.142018,1.030389,1.803624,0.511930,-0.532412,11.532230,0
1,-0.006558,15.830119,-0.877863,-0.665609,1.939337,2.184802,-0.654894,-1.818208,-2.083371,-1.755544,...,0.705715,-0.611346,-0.256529,-0.132202,0.315106,1.087893,-0.264199,0.729070,11.837788,0
2,1.621829,0.022880,0.628173,-0.665609,1.944366,2.003829,-0.759966,-0.294276,0.071056,0.076887,...,1.590999,-0.718081,-0.275236,-0.133737,-0.373767,0.220560,-0.403778,0.166637,11.993986,0
3,1.458537,0.098404,0.478169,1.502383,0.418686,0.424192,-0.692850,-0.377448,-0.069435,0.044057,...,-0.411254,-0.482125,-0.309781,-0.130670,-0.381454,0.319395,-0.235043,-0.434613,11.899311,0
4,1.378403,0.708624,-0.217847,1.502383,0.304383,0.271906,-0.681392,-1.084413,-0.898617,-0.568914,...,-0.508870,-0.487276,0.293447,-0.132631,0.790557,1.576119,1.011800,0.414773,12.127791,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13983,-1.027253,-0.295771,-0.394442,-0.665609,-0.573781,-0.518446,0.752327,-0.023711,0.115461,0.042379,...,0.848428,0.906560,-0.389164,-0.125772,-0.550939,-0.169418,-0.388528,-0.570823,11.429912,1
13984,0.641584,-0.648362,-2.575294,1.502383,-0.195789,-0.083045,-0.445079,-3.791876,-5.886361,-5.370951,...,0.280180,-0.186890,1.139096,-0.135678,0.314260,-0.132503,-0.096975,-0.469805,11.647264,1
13985,0.954400,-0.648362,-6.762250,1.502383,1.239465,1.130181,-0.707778,-2.491184,-14.103761,-11.022615,...,1.055706,-0.702964,10.579563,-0.153648,0.016519,-0.171411,0.030245,-0.414785,11.677890,1
13986,0.340726,-0.090336,-0.575757,-0.665609,-0.311726,-0.283600,-0.337233,0.072408,-0.036223,0.059226,...,-0.286733,-0.296554,-0.424347,-0.153890,-0.224048,2.342012,-0.372922,-0.453420,11.599441,1


In [35]:
# H가 1인 경우 이분산성 / H가 0인 경우 등분산

def bartlett_test_2(col, p_value = 0.05, H = 1):
    list= []
    for i in col:
        T, p_val =bartlett(x_등분산[x_등분산['부실']==1][i], x_등분산[x_등분산['부실']==0][i]) 
        list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    if H == 1:
        a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    else:
        a = list[(list['p_value'] >= p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
# 이분산성 변수
x_hetero = bartlett_test_2(X_train_MM.columns, H = 1)
# 등분산성 변수
x_homo = bartlett_test_2(X_train_MM.columns, H = 0)
set_이분산 = set(x_hetero["변수"])
set_등분산 = set(x_homo["변수"])

In [36]:
# 2) t_test
def t_test_2(col, col_h0, col_h1, p_value = 0.05):
    list= []
    for i in col:
        if (col_h0['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(x_등분산[x_등분산['부실']==1][i], x_등분산[x_등분산['부실']==0][i], equal_var=True) # 등분산성 : wald t-test
            list.append([i, p_val])
        elif (col_h1['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(x_등분산[x_등분산['부실']==1][i], x_등분산[x_등분산['부실']==0][i], equal_var=False) # 이분산성 : welch’s t-test
            list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
    return a

x_ttest = t_test_2(X_train_MM.columns, x_homo, x_hetero, p_value=0.05)
print("유의한 피쳐 수 :", len(x_ttest))
selected_result = x_ttest.sort_values(by="변수", ascending=True)
list(selected_result['변수'])

유의한 피쳐 수 : 36


['DOL',
 'EBTA',
 'EV/EBITDA',
 'OM',
 'R&D비율',
 'RETA',
 '거래량증가율',
 '거래량회전율',
 '고정비율',
 '금융비용부담률',
 '당좌비율',
 '로그시가총액',
 '매출액순이익률',
 '매출액정상영업이익률',
 '매출액증가율',
 '부채비율',
 '설비투자효율',
 '수정거래량',
 '순운전자본비율',
 '순이익증가율',
 '시가총액증가율',
 '영업이익증가율',
 '영업활동으로 인한 현금흐름',
 '유동비율',
 '유동자산증가율',
 '유형자산증가율',
 '이자보상배율(이자비용)',
 '자기자본구성비율',
 '자기자본순이익률',
 '재고자산회전률',
 '종업원수증가율',
 '채무부담비율',
 '총자본정상영업이익률',
 '총자본증가율',
 '총자본투자효율',
 '총자본회전률']

In [37]:
# stepwise
def stepwise_feature_selection(feature, target, variables):
    import statsmodels.api as sm
    import matplotlib.pyplot as plt
    import warnings
    import pandas as pd
    warnings.filterwarnings("ignore")
    
    y = target ## 반응 변수

    selected_variables = [] ## 선택된 변수들
    sl_enter = 0.1
    sl_remove = 0.1
    
    sv_per_step = [] ## 각 스텝별로 선택된 변수들
    r_squared = [] ## 각 스텝별 수정된 결정계수
    steps = [] ## 스텝
    step = 0
    while len(variables) > 0:
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) ## 변수의 p-value
        ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서
        ## 선형 모형을 적합한다.
        for col in remainder: 
            X = feature[selected_variables+[col]]
            X = sm.add_constant(X)
            model = sm.Logit(y,X).fit(disp=0)
            pval[col] = model.pvalues[col]
    
        min_pval = pval.min()
        if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
            selected_variables.append(pval.idxmin())
            ## 선택된 변수들에대해서
            ## 어떤 변수를 제거할지 고른다.
            while len(selected_variables) > 0:
                selected_X = feature[selected_variables]
                selected_X = sm.add_constant(selected_X)
                selected_pval = sm.Logit(y,selected_X).fit(disp=0).pvalues[1:]
                max_pval = selected_pval.max()
                if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                    remove_variable = selected_pval.idxmax()
                    selected_variables.remove(remove_variable)
                else:
                    break
            
            step += 1
            steps.append(step)
            presudo_r_squared = sm.Logit(y,sm.add_constant(feature[selected_variables])).fit(disp=0).prsquared
            r_squared.append(presudo_r_squared)
            sv_per_step.append(selected_variables.copy())
        else:
            break
      
    return selected_variables
    
selected_result_1 = stepwise_feature_selection(X_train_MM, Y_train, X_train_MM.columns.tolist())
selected_result_1

['EBTA',
 '금융비용부담률',
 '부채비율',
 '수정거래량',
 'DOL',
 '자기자본순이익률',
 'RETA',
 '총자본회전률',
 '로그시가총액',
 '매출채권회전률',
 'EV/EBITDA',
 '거래량증가율',
 '당좌비율',
 '매출액순이익률',
 '채무부담비율',
 '이자보상배율(이자비용)',
 'DFL',
 '종업원수증가율',
 '고정비율',
 '유동자산증가율',
 '총자본증가율',
 '재고자산회전률',
 '유동비율',
 '영업이익증가율',
 '순이익증가율',
 '설비투자효율',
 '매출액증가율']

In [38]:
lasso_model = LogisticRegression()
param_grid = {'penalty' : ['l1'], 
                'C' : np.logspace(-4, 4, 20),
                'solver' : ['liblinear', 'saga']}

grid_search = GridSearchCV(lasso_model, param_grid=param_grid, return_train_score=True, cv=5)
grid_search.fit(X_train_MM, Y_train)

df_grid = pd.DataFrame(grid_search.cv_results_)
df_grid = df_grid.sort_values(by=['rank_test_score'], ascending=True)
df_grid[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]
print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_search.best_score_))

GridSearchCV 최적 파라미터: {'C': 0.23357214690901212, 'penalty': 'l1', 'solver': 'saga'}
GridSearchCV 최고 정확도:0.8571


In [39]:
# lasso_best = LogisticRegression(C=0.012742749857031334, penalty='l1', solver='liblinear').fit(X_train_MM, Y_train)
lasso_best = LogisticRegression(**grid_search.best_params_).fit(X_train_MM, Y_train)
selected_result_2 = pd.DataFrame()
selected_result_2['feature'] = X_train_MM.columns
selected_result_2['coef'] = lasso_best.coef_[0]
selected_result_2.drop(selected_result_2[selected_result_2['coef']==0].index, inplace=True)
list(selected_result_2['feature'])

['자기자본구성비율',
 '설비투자효율',
 '총자본투자효율',
 '이자보상배율(이자비용)',
 '유동비율',
 '당좌비율',
 '부채비율',
 '총자본정상영업이익률',
 '매출액순이익률',
 '자기자본순이익률',
 '매출채권회전률',
 '재고자산회전률',
 '총자본회전률',
 '매출액증가율',
 '총자본증가율',
 '유동자산증가율',
 '유형자산증가율',
 '영업이익증가율',
 '순이익증가율',
 'RETA',
 'EBTA',
 'OM',
 '종업원수증가율',
 'DOL',
 'DFL',
 'EV/EBITDA',
 '영업활동으로 인한 현금흐름',
 '금융비용부담률',
 '고정비율',
 'R&D비율',
 '채무부담비율',
 '거래량회전율',
 '수정거래량',
 '거래량증가율',
 '시가총액증가율',
 '로그시가총액']

In [40]:
list_1 = list(selected_result['변수'])
list_2 = list(selected_result_1)
list_3 = list(selected_result_2['feature'])
list_col_all = Raw.columns

In [41]:
def func_1(x):
    if x in list_1:
        return 1
    else:
        return 0

def func_2(x):
    if x in list_2:
        return 1
    else:
        return 0

def func_3(x):
    if x in list_3:
        return 1
    else:
        return 0


In [42]:
feature_counts = pd.DataFrame()
feature_counts['feature'] = list_col_all
feature_counts['1'] = list_col_all.map(func_1)
feature_counts['2'] = list_col_all.map(func_2)
feature_counts['3'] = list_col_all.map(func_3)

feature_counts["total"] = feature_counts["1"]+feature_counts["2"]+feature_counts["3"]
feature_final = feature_counts[feature_counts["total"]>=2]
list_feature_final = list(feature_final["feature"])
print("선택된 피쳐수 :", len(list_feature_final))
feature_final

선택된 피쳐수 : 36


,feature,1,2,3,total
3,자기자본구성비율,1,0,1,2
4,설비투자효율,1,1,1,3
5,총자본투자효율,1,0,1,2
6,이자보상배율(이자비용),1,1,1,3
7,유동비율,1,1,1,3
8,당좌비율,1,1,1,3
9,부채비율,1,1,1,3
10,총자본정상영업이익률,1,0,1,2
12,매출액순이익률,1,1,1,3
13,자기자본순이익률,1,1,1,3


In [43]:
select_final = []
for i in feature_final.index:
    if feature_final.loc[i, "total"] >= 3:
        select_final.append(feature_final.loc[i, 'feature'])
select_final


['설비투자효율',
 '이자보상배율(이자비용)',
 '유동비율',
 '당좌비율',
 '부채비율',
 '매출액순이익률',
 '자기자본순이익률',
 '재고자산회전률',
 '총자본회전률',
 '매출액증가율',
 '총자본증가율',
 '유동자산증가율',
 '영업이익증가율',
 '순이익증가율',
 'RETA',
 'EBTA',
 '종업원수증가율',
 'DOL',
 'EV/EBITDA',
 '금융비용부담률',
 '고정비율',
 '채무부담비율',
 '로그시가총액',
 '수정거래량',
 '거래량증가율']

In [47]:
X_train = X_train_MM[select_final]
X_test = X_test_MM[select_final]
X_train.to_csv('./csv/X_train.csv', index=0)
X_test.to_csv('./csv/X_test.csv', index=0)
Y_train.to_csv('./csv/Y_train.csv', index=0)
Y_test.to_csv('./csv/Y_test.csv', index=0)